# FIN3080 Assignment 4

> **Author Information**:
>- Name: `Weiyuan Xu`
>- StdID: `120090888`

> **Reference**:
>- Chen, Yifan et al.(2019). “Empirical test of CAPM in Shanghai securities market”. In: Finance 9, pp.28–33.


## 0. Preparations

### 0-1. Load in packages

In [47]:
import pandas as pd
import numpy as np
import datetime as dt
import math

import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)
import statsmodels.api as sm

from mpl_toolkits.mplot3d import Axes3D

### 0-2. Load in datasets

#### 0-2-1. Read datasets

> **Data Resources**:
>- Data Service Platform: `CSMAR`
>- Databases: `CSMAR_China Stock Market Series`

**Detailed Selection Process of Data Tables Used:**
- table_one: China Stock Market Series -> Stock Trading -> Individual Stock Trading -> Weekly Stock Price & Returns
- table_two: China Stock Market Series -> Stock Trading -> Individual Stock Trading -> Weekly Stock Price & Returns
- table_three: China Stock Market Series -> Stock Trading -> Market Trading -> Weekly Market Returns
- table_four: China Stock Market Series -> Stock Trading -> Exchange Rate & Interest Rate -> Risk-Free Rate

***Note: All above tables only include the information of A-shares with the filtering function of CSMAR***

In [48]:
table_one = pd.read_csv('1.csv')
table_two = pd.read_csv('2.csv')
table_three = pd.read_csv('3.csv')
table_four = pd.read_csv('4.csv')

#### 0-2-2. Preliminary processing and interpret for individual datasets

***Market Type Extraction***
- Main Board: ***Market Type*** = 1, 4

**Variables in Table 1:**
- Weekly Return
- Market Type

**Interpretation of picking corresponding tickers:**
- Weekly Return: Use ***Weekly Return with Cash Dividend Reinvested***.


$$Net Return = \frac{Price_{t+1} + C}{Price_t} = \frac{Price'_{t+1}}{Price_t}$$
<font color='blue'>Only when we incorporate the cash dividend reinvestment to our price, then we can compute the return more accurately.</font>


- Market Type: Use ***Market Type*** to classify different markets.

In [49]:
table_one = table_one.rename(columns={"Stkcd": "Stock_Code", "Trdwnt": "Week", "Wretwd": "Weekly_Individual_Return", "Markettype": "Market_Type"})
table_one = table_one.loc[table_one["Market_Type"].isin([1, 4])]
table_one.head()

,Stock_Code,Week,Weekly_Individual_Return,Market_Type
0,603060,2017-20,0.028034,1
1,603060,2017-21,-0.135018,1
2,603060,2017-22,-0.047674,1
3,603060,2017-23,0.042390,1
4,603060,2017-24,-0.001936,1


**Variables in Table 2:**
- Weekly Return
- Market Type

**Interpretation of picking corresponding tickers:**
- Weekly Return: Use ***Weekly Return with Cash Dividend Reinvested***.
- Market Type: Use ***Market Type*** to classify different markets.

In [50]:
table_two = table_two.rename(columns={"Stkcd": "Stock_Code", "Trdwnt": "Week", "Wretwd": "Weekly_Individual_Return", "Markettype": "Market_Type"})
table_two = table_two.loc[table_two["Market_Type"].isin([1, 4])]
table_two.head()

,Stock_Code,Week,Weekly_Individual_Return,Market_Type
0,1,2017-01,0.003297,4
1,1,2017-02,0.003286,4
2,1,2017-03,0.006550,4
3,1,2017-04,0.011931,4
4,1,2017-05,-0.007503,4


**Variables in Table 3:**
- Weekly Market Return
- Market Capitalization
- Market Type

**Interpretation of picking corresponding tickers:**
- Weekly Market Return: Use ***Weekly Market Return With Cash Dividend Reinvested (Total-Value-Weighted)*** since 
    - Compared with ***Weekly Market Return With Cash Dividend Reinvested (Equally-Weighted)***, ***Weekly Market Return With Cash Dividend Reinvested (Total-Value-Weighted)*** takes market capitalization of different individual stocks into consideration.
    - Compared with ***Weekly Market Return With Cash Dividend Reinvested (Current-Value-Weighted / Negotiable-Value-Weighted)***, ***Weekly Market Return With Cash Dividend Reinvested (Total-Value-Weighted)*** incorporates all the outstanding shares rather than just tradable shares, which is more representative.
- Market Capitalization: Use ***Market Capitalization*** to compute the weighted-average Weekly Market Return for the whole main board.
- Market Type: Use ***Market Type*** to classify different markets.

In [51]:
table_three = table_three.rename(columns={"Markettype": "Market_Type", "Trdwnt": "Week", "Wretwdtl": "Weekly_Market_Return_temp", "Wmvttl": "Market_Capitalization"})
table_three.head()

,Market_Type,Week,Weekly_Market_Return_temp,Market_Capitalization
0,2,2017-01,0.005365,1.851174e+07
1,1,2017-01,0.017187,2.811745e+10
2,4,2017-01,0.012805,1.618719e+10
3,8,2017-01,0.004803,1.065178e+08
4,16,2017-01,0.003144,4.881104e+09


**Variable in Table 4:**
- Weekly Risk Free Rate

**Interpretation of picking corresponding ticker:**
- Weekly Risk Free Rate: Use ***Weeklized Risk-Free Rate*** since these two are equivalent.

In [52]:
table_four = table_four.rename(columns={"Clsdt": "Date", "Nrrwkdt": "Rf"})
table_four = table_four.loc[:,["Date", "Rf"]]
table_four.head()

,Date,Rf
0,2017-01-01,0.0286
1,2017-01-02,0.0286
2,2017-01-03,0.0286
3,2017-01-04,0.0286
4,2017-01-05,0.0286


#### 0-2-3. Further prepare for the data

##### 0-2-3-1. Construct individual returns

In [53]:
df_individual = pd.concat([table_one, table_two], axis = 0)
df_individual = df_individual.sort_values(by = ['Stock_Code', "Week"])
df_individual = df_individual.reset_index()
df_individual = df_individual.loc[:,["Stock_Code", "Week", "Weekly_Individual_Return"]]
df_individual.head()

,Stock_Code,Week,Weekly_Individual_Return
0,1,2017-01,0.003297
1,1,2017-02,0.003286
2,1,2017-03,0.006550
3,1,2017-04,0.011931
4,1,2017-05,-0.007503


In [54]:
df_individual.describe()

,Stock_Code,Weekly_Individual_Return
count,880097.000000,879303.000000
mean,311967.852883,0.001019
std,299726.061789,0.064852
min,1.000000,-0.895775
25%,2270.000000,-0.032099
50%,600067.000000,-0.001692
75%,600968.000000,0.028355
max,605599.000000,3.872825


##### 0-2-3-2. Construct market returns

In [55]:
df_market = table_three.loc[table_three["Market_Type"].isin([1, 4])]
df_market.head()

,Market_Type,Week,Weekly_Market_Return_temp,Market_Capitalization
1,1,2017-01,0.017187,2.811745e+10
2,4,2017-01,0.012805,1.618719e+10
6,1,2017-02,-0.013058,2.780168e+10
7,4,2017-02,-0.031853,1.573545e+10
11,1,2017-03,0.003825,2.795989e+10


In [56]:
weighted_avg = (df_market['Weekly_Market_Return_temp'] * df_market['Market_Capitalization']).groupby(df_market['Week']).sum() / df_market['Market_Capitalization'].groupby(df_market['Week']).sum()
df_market = pd.DataFrame({'Weekly_Market_Return': weighted_avg})
df_market = df_market.reset_index()
df_market.head()

,Week,Weekly_Market_Return
0,2017-01,0.015586
1,2017-02,-0.019851
2,2017-03,-0.003306
3,2017-04,0.014733
4,2017-05,-0.005345


In [57]:
df_market.describe()

,Weekly_Market_Return
count,308.000000
mean,0.000709
std,0.024138
min,-0.095803
25%,-0.011921
50%,0.001221
75%,0.016979
max,0.083595


##### 0-2-3-3. Construct risk-free returns

In [58]:
df_riskfree = table_four
df_riskfree.describe()

,Rf
count,2.191000e+03
mean,2.860000e-02
std,1.006369e-15
min,2.860000e-02
25%,2.860000e-02
50%,2.860000e-02
75%,2.860000e-02
max,2.860000e-02


**Remarks:**
It is shown that weekly risk-free rates are all 0.0286%.

In [59]:
df_riskfree["Week"] = df_market["Week"]
df_riskfree = df_riskfree.loc[:, ["Week"]]
df_riskfree["Rf"] = 0.000286
df_riskfree.head()


,Week,Rf
0,2017-01,0.000286
1,2017-02,0.000286
2,2017-03,0.000286
3,2017-04,0.000286
4,2017-05,0.000286


In [60]:
df_riskfree.describe()

,Rf
count,2.191000e+03
mean,2.860000e-04
std,6.344031e-18
min,2.860000e-04
25%,2.860000e-04
50%,2.860000e-04
75%,2.860000e-04
max,2.860000e-04


## 1. Step 1: Divide the Data into Three Periods

**Dividing Criterion:**
Since our data incorporate 6 years, then we divide them into three parts based on year approximately.

### 1-1. Divide for individual returns

In [61]:
df_individual_one = df_individual[df_individual['Week'].str.startswith('2017') | df_individual['Week'].str.startswith('2018')]
df_individual_two = df_individual[df_individual['Week'].str.startswith('2019') | df_individual['Week'].str.startswith('2020')]
df_individual_three = df_individual[df_individual['Week'].str.startswith('2021') | df_individual['Week'].str.startswith('2022')]

In [62]:
df_individual_one.head()

,Stock_Code,Week,Weekly_Individual_Return
0,1,2017-01,0.003297
1,1,2017-02,0.003286
2,1,2017-03,0.006550
3,1,2017-04,0.011931
4,1,2017-05,-0.007503


### 1-2. Divide for market returns

In [63]:
df_market_one = df_market[df_market['Week'].str.startswith('2017') | df_market['Week'].str.startswith('2018')]
df_market_two = df_market[df_market['Week'].str.startswith('2019') | df_market['Week'].str.startswith('2020')]
df_market_three = df_market[df_market['Week'].str.startswith('2021') | df_market['Week'].str.startswith('2022')]

In [64]:
df_market_one.head()

,Week,Weekly_Market_Return
0,2017-01,0.015586
1,2017-02,-0.019851
2,2017-03,-0.003306
3,2017-04,0.014733
4,2017-05,-0.005345


### 1-3. Divide for risk-free returns

In [65]:
df_riskfree_one = df_riskfree[df_riskfree['Week'].str.startswith('2017') | df_riskfree['Week'].str.startswith('2018')]
df_riskfree_two = df_riskfree[df_riskfree['Week'].str.startswith('2019') | df_riskfree['Week'].str.startswith('2020')]
df_riskfree_three = df_riskfree[df_riskfree['Week'].str.startswith('2021') | df_riskfree['Week'].str.startswith('2022')]

In [66]:
df_riskfree_one.head()

,Week,Rf
0,2017-01,0.000286
1,2017-02,0.000286
2,2017-03,0.000286
3,2017-04,0.000286
4,2017-05,0.000286


## 2. Step 2: Time-Series Regression for the First-Period Data

In [67]:
df_excessMKTRet_one = pd.merge(df_market_one, df_riskfree_one, on = ['Week'], how = 'left')
df_excessMKTRet_one["Excess_Market_Return"] = df_excessMKTRet_one["Weekly_Market_Return"] - df_excessMKTRet_one["Rf"]
df_excessMKTRet_one = df_excessMKTRet_one.loc[:,["Week", "Excess_Market_Return"]]
df_excessMKTRet_one.head()

,Week,Excess_Market_Return
0,2017-01,0.015300
1,2017-02,-0.020137
2,2017-03,-0.003592
3,2017-04,0.014447
4,2017-05,-0.005631


In [68]:
df_2 = pd.merge(df_individual_one, df_riskfree_one, on = ['Week'], how = 'left')
df_2["Excess_Individual_Return"] = df_2["Weekly_Individual_Return"] - df_2["Rf"]
df_2 = pd.merge(df_2, df_excessMKTRet_one, on = ['Week'], how = 'left')
df_2 = df_2.loc[:,["Stock_Code", "Week", "Excess_Individual_Return", "Excess_Market_Return"]]
df_2.head()

,Stock_Code,Week,Excess_Individual_Return,Excess_Market_Return
0,1,2017-01,0.003011,0.015300
1,1,2017-02,0.003000,-0.020137
2,1,2017-03,0.006264,-0.003592
3,1,2017-04,0.011645,0.014447
4,1,2017-05,-0.007789,-0.005631


In [69]:
def reg(x, y):
    X = sm.add_constant(x)
    result = sm.OLS(y, X).fit()
    return result

beta_one = df_2.groupby('Stock_Code').apply(lambda x: reg(x.Excess_Market_Return, x.Excess_Individual_Return).params['Excess_Market_Return'])
beta_one = pd.DataFrame(beta_one)
beta_one = beta_one.reset_index()
beta_one.columns = ['Stock_Code','Individual_Beta']
beta_one.head()

,Stock_Code,Individual_Beta
0,1,1.008934
1,2,1.275752
2,4,0.434109
3,5,1.148236
4,6,1.739789


In [70]:
beta_one.describe()

,Stock_Code,Individual_Beta
count,2835.000000,2460.000000
mean,307727.850794,1.156060
std,299845.052650,0.336985
min,1.000000,-1.119022
25%,2242.500000,0.959895
50%,600036.000000,1.160074
75%,600869.500000,1.363563
max,603999.000000,4.193109


**Remarks:**
It is obvious that there are some null values for some certain stocks, we need to drop it before grouping.

In [71]:
beta_one = beta_one.dropna()
beta_one.describe()

,Stock_Code,Individual_Beta
count,2460.000000,2460.000000
mean,287574.471138,1.156060
std,299435.758844,0.336985
min,1.000000,-1.119022
25%,2150.750000,0.959895
50%,2774.000000,1.160074
75%,600677.250000,1.363563
max,603999.000000,4.193109


## 3. Step 3: Construct Portfolios based on Beta Values in the First Period and Regress for Portfolio Returns

### 3-1. Construct Portfolios based on Beta Values in the First Period

In [72]:
beta_one = beta_one.sort_values(by = 'Individual_Beta')
beta_one['Group'] = pd.qcut(beta_one['Individual_Beta'], 10, labels = False) + 1
beta_one = beta_one.sort_values(by = ['Group', 'Individual_Beta'])
beta_one.head()

,Stock_Code,Individual_Beta,Group
247,693,-1.119022,1
332,816,-0.323448,1
1391,600005,-0.124671,1
950,2485,-0.075683,1
1957,600687,-0.055983,1


### 3-2. Implement Time-Series Regression

In [73]:
df_excessMKTRet_two = pd.merge(df_market_two, df_riskfree_two, on = ['Week'], how = 'left')
df_excessMKTRet_two["Excess_Market_Return"] = df_excessMKTRet_two["Weekly_Market_Return"] - df_excessMKTRet_two["Rf"]
df_excessMKTRet_two = df_excessMKTRet_two.loc[:,["Week", "Excess_Market_Return"]]
df_excessMKTRet_two.head()

,Week,Excess_Market_Return
0,2019-01,0.008878
1,2019-02,0.019406
2,2019-03,0.014006
3,2019-04,0.001112
4,2019-05,0.002665


In [74]:
df_3 = pd.merge(df_individual_two, df_riskfree_two, on = ['Week'], how = 'left')
df_3["Excess_Individual_Return"] = df_3["Weekly_Individual_Return"] - df_3["Rf"]
df_3 = df_3.loc[:,["Stock_Code", "Week", "Excess_Individual_Return"]]
df_3.head()

,Stock_Code,Week,Excess_Individual_Return
0,1,2019-01,0.039160
1,1,2019-02,0.045868
2,1,2019-03,0.022263
3,1,2019-04,0.054364
4,1,2019-05,0.017896


In [75]:
df_3 = df_3.dropna()
df_3 = pd.merge(df_3, beta_one, on = ['Stock_Code'], how = 'left')
df_3 = df_3.groupby(['Group', 'Week'])['Excess_Individual_Return'].mean()
df_3 = pd.DataFrame(df_3)
df_3 = df_3.reset_index()
df_3 = df_3.rename(columns={'Excess_Individual_Return': 'Excess_Group_Return'})
df_3["Group"] = df_3["Group"].astype(int)
df_3 = df_3.dropna()
df_3.head()


,Group,Week,Excess_Group_Return
0,1,2019-01,0.001728
1,1,2019-02,0.016531
2,1,2019-03,-0.000283
3,1,2019-04,-0.007133
4,1,2019-05,-0.028198


**Remarks:**
Here, the group returns' computation obeys the criterion used in the paper (Equal-weighted Average).

In [76]:
df_3 = pd.merge(df_3, df_excessMKTRet_two, on = ['Week'], how = 'left')
df_3.head()


,Group,Week,Excess_Group_Return,Excess_Market_Return
0,1,2019-01,0.001728,0.008878
1,1,2019-02,0.016531,0.019406
2,1,2019-03,-0.000283,0.014006
3,1,2019-04,-0.007133,0.001112
4,1,2019-05,-0.028198,0.002665


In [77]:
df_3.describe()

,Group,Excess_Group_Return,Excess_Market_Return
count,1040.000000,1040.000000,1040.000000
mean,5.500000,0.003594,0.004394
std,2.873663,0.031197,0.026509
min,1.000000,-0.088332,-0.060101
25%,3.000000,-0.014993,-0.009375
50%,5.500000,0.001467,0.004943
75%,8.000000,0.023500,0.019164
max,10.000000,0.096925,0.083309


In [78]:
beta_two = df_3.groupby('Group').apply(lambda x: reg(x.Excess_Market_Return, x.Excess_Group_Return).params['Excess_Market_Return'])
beta_two = pd.DataFrame(beta_two)
beta_two = beta_two.reset_index()
beta_two.columns = ['Group','Group_Beta']
beta_two.head()

,Group,Group_Beta
0,1,0.933000
1,2,0.980540
2,3,0.998030
3,4,1.070760
4,5,1.114878


In [79]:
df_3 = df_3.groupby('Group').apply(lambda x: sm.OLS(x['Excess_Group_Return'], sm.add_constant(x['Excess_Market_Return'])).fit())
    
alpha = df_3.apply(lambda x: x.params[0])
alpha_t = df_3.apply(lambda x: x.tvalues[0])
alpha_p = df_3.apply(lambda x: x.pvalues[0])
beta = df_3.apply(lambda x: x.params[1])
beta_t = df_3.apply(lambda x: x.tvalues[1])
beta_p = df_3.apply(lambda x: x.pvalues[1])
rsquared = df_3.apply(lambda x: x.rsquared)

df_3 = pd.concat([alpha, alpha_t, alpha_p, beta, beta_t, beta_p, rsquared], axis = 1)
df_3.columns = ['αp', 'αp的t值', '显著性', 'βp', 'βp的t值', '显著性', 'R-squared']
df_3.index.name = '组合名称'

df_3.head(10)

,αp,αp的t值,显著性,βp,βp的t值,显著性,R-squared
组合名称,,,,,,,
1,-0.003161,-3.126657,0.002304,0.933000,24.789105,5.647082e-45,0.857641
2,-0.001235,-1.409426,0.161751,0.980540,30.059634,1.716546e-52,0.898566
3,-0.000971,-0.953381,0.342651,0.998030,26.308512,2.949400e-47,0.871559
4,-0.000728,-0.725422,0.469854,1.070760,28.651638,1.370310e-50,0.889481
5,-0.000891,-0.892345,0.374309,1.114878,29.996453,2.081998e-52,0.898182
6,-0.000479,-0.451716,0.652433,1.124066,28.480757,2.358562e-50,0.888299
7,-0.000876,-0.793891,0.429102,1.117396,27.204491,1.477451e-48,0.878872
8,-0.001092,-0.881953,0.379876,1.171259,25.407182,6.478497e-46,0.863550
9,-0.000735,-0.556441,0.579128,1.171006,23.802675,1.940793e-43,0.847435


**Analysis:**

- **Analysis for Beta**: As can be seen from Table 2, the βp values of all portfolios are similar, mostly around 1, and the significance level is basically small, which indicates that stock returns are significantly affected by stock market returns. 

- **Analysis for Alpha**: In addition, all the αp values are close to 0. However, 90% of them are not significant (only the first one is significant), which means that the null hypothesis: existence of alpha return is not significantly rejected.

- **Analysis for the whole Regression**: At the same time, R-square does not increase with the increase of βp value, which indicates that stock returns may be affected by other factors besides systemic risk.

## 4. Step 4: Cross-Sectional Regression for CAPM

### 4-1. Compute Alphas for the Third-Period Data

In [80]:
df_4 = pd.merge(df_individual_three, df_riskfree_three, on = ['Week'], how = 'left')
df_4["Excess_Individual_Return"] = df_4["Weekly_Individual_Return"] - df_4["Rf"]
df_4 = df_4.loc[:,["Stock_Code", "Week", "Excess_Individual_Return"]]
df_4.head()

,Stock_Code,Week,Excess_Individual_Return
0,1,2021-02,0.026084
1,1,2021-03,0.057649
2,1,2021-04,0.048762
3,1,2021-05,0.047830
4,1,2021-06,0.079402


In [81]:
df_4 = pd.merge(df_4, beta_one, on = ['Stock_Code'], how = 'left')
df_4 = df_4.groupby(['Week', 'Group'])['Excess_Individual_Return'].mean()
df_4 = pd.DataFrame(df_4)
df_4 = df_4.reset_index()
df_4 = df_4.rename(columns = {'Excess_Individual_Return': 'Excess_Group_Return'})
df_4["Group"] = df_4["Group"].astype(int)
df_4 = df_4.dropna()
df_4.head()

df_temp = df_4

In [82]:
df_temp.head()

,Week,Group,Excess_Group_Return
0,2021-02,1,-0.022499
1,2021-02,2,-0.004403
2,2021-02,3,-0.017068
3,2021-02,4,-0.013665
4,2021-02,5,-0.014925


In [83]:
AVG_Excess_Group_Return = df_4.groupby("Group")["Excess_Group_Return"].mean()
df_4 = pd.DataFrame(AVG_Excess_Group_Return).reset_index()
df_4.columns = ['Group', 'AVG_Excess_Group_Return']
df_4.head(10)

y_observed = df_4['AVG_Excess_Group_Return'] + 0.000286

In [84]:
df_4.describe()

,Group,AVG_Excess_Group_Return
count,10.00000,10.000000
mean,5.50000,0.002020
std,3.02765,0.000303
min,1.00000,0.001392
25%,3.25000,0.001876
50%,5.50000,0.002005
75%,7.75000,0.002191
max,10.00000,0.002447


### 4-2. Implement Cross-Sectional Regression

In [85]:
df_3 = df_3.reset_index()
df_3 = df_3.rename(columns={"组合名称": "Group"})
df_3 = pd.merge(df_3, df_4, on = "Group", how = 'left')
df_3.head(10)

,Group,αp,αp的t值,显著性,βp,βp的t值,显著性,R-squared,AVG_Excess_Group_Return
0,1,-0.003161,-3.126657,0.002304,0.933000,24.789105,5.647082e-45,0.857641,0.002108
1,2,-0.001235,-1.409426,0.161751,0.980540,30.059634,1.716546e-52,0.898566,0.001857
2,3,-0.000971,-0.953381,0.342651,0.998030,26.308512,2.949400e-47,0.871559,0.001392
3,4,-0.000728,-0.725422,0.469854,1.070760,28.651638,1.370310e-50,0.889481,0.001981
4,5,-0.000891,-0.892345,0.374309,1.114878,29.996453,2.081998e-52,0.898182,0.002390
5,6,-0.000479,-0.451716,0.652433,1.124066,28.480757,2.358562e-50,0.888299,0.001841
6,7,-0.000876,-0.793891,0.429102,1.117396,27.204491,1.477451e-48,0.878872,0.001934
7,8,-0.001092,-0.881953,0.379876,1.171259,25.407182,6.478497e-46,0.863550,0.002218
8,9,-0.000735,-0.556441,0.579128,1.171006,23.802675,1.940793e-43,0.847435,0.002029
9,10,-0.001997,-1.417472,0.159393,1.266914,24.155090,5.421063e-44,0.851197,0.002447


In [86]:
X = sm.add_constant(df_3["βp"])
y = df_3["AVG_Excess_Group_Return"]
ols_model = sm.OLS(y, X).fit()

ols_model.summary()

/Users/a86139/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1769: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     AVG_Excess_Group_Return   R-squared:                       0.317
Model:                                 OLS   Adj. R-squared:                  0.232
Method:                      Least Squares   F-statistic:                     3.718
Date:                     Wed, 26 Apr 2023   Prob (F-statistic):             0.0900
Time:                             15:13:30   Log-Likelihood:                 69.251
No. Observations:                       10   AIC:                            -134.5
Df Residuals:                            8   BIC:                            -133.9
Df Model:                                1                                         
Covariance Type:                 nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0002      0.001      0.178      0.863      -0.002       0.002
βp             0.0017      0.001      1.928      0.090      -0.000       0.004
==============================================================================
Omnibus:                        0.200   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.905   Jarque-Bera (JB):                0.174
Skew:                          -0.203   Prob(JB):                        0.917
Kurtosis:                       2.498   Cond. No.                         22.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [87]:
γ0 = [0.0002, 0.178]
γ1 = [0.0017, 1.928]
R_squared = [0.317, None]
F_statistic = [3.718, None]
P = [0.0900, None]

data = {'γ0': γ0, 'γ1': γ1, 'R-squared': R_squared, 
        'F-statistic': F_statistic, 'P': P}
df_result = pd.DataFrame(data)
df_result = df_result.rename(index={0:'系数', 1:'t检验值'})
df_result.head()

,γ0,γ1,R-squared,F-statistic,P
系数,0.0002,0.0017,0.317,3.718,0.09
t检验值,0.1780,1.9280,NaN,NaN,NaN


**Analysis:**

- **Analysis for γ0 Coefficient:** As can be seen from Table 3, the γ0 value is close to 0; however, its p-value is 0.863 (which is higher than 0.1), then it means that the constant γ0 is not significant to be zero, the null hypothesis is not rejected, indicating there may be the presence of factors other than systemic risk.

- **Analysis for γ1 Coefficient:** In addition, the γ1 value is positive (0.0017); and its p-value is 0.09 (which is smaller than 0.1), then it means that γ1 is significant to be positive, showing that there is a significant positive correlation between return and systemic risk, which is consistent with CAPM. 

- **Analysis for the whole Regression:** At the same time, R-square is 0.317, which is relatively low, which means the fitting result is not sufficiently great. However, the p-value of the regression is 0.09 (which is smaller than 0.1), then it means that the model does make sense.

In [102]:
ticker = [1, 2, 999]
num = [200, 300, 1000]
price = [24.1, 26.7, 1.8]
weight = [0.012, 0.018, 0.023]

data = {'ticker': ticker, 'num': num, 'price': price, 
        'weight': weight}
holding = pd.DataFrame(data)
holding.head()



,ticker,num,price,weight
0,1,200,24.1,0.012
1,2,300,26.7,0.018
2,999,1000,1.8,0.023


In [114]:
# # a ❌
# holding = (holding.set_index('ticker'))
# weight = holding.loc[['1','999'],'weight']

# a 对
import pandas as pd

ticker = [1, 2, 999]
num = [200, 300, 1000]
price = [24.1, 26.7, 1.8]
weight = [0.012, 0.018, 0.023]

data = {'ticker': ticker, 'num': num, 'price': price, 
        'weight': weight}
holding = pd.DataFrame(data)

# Set index to 'ticker'
holding = holding.set_index('ticker')

# Select rows using label '1' and '999'
weights = holding.loc[[1, 999], 'weight']
print(weights)

ticker
1      0.012
999    0.023
Name: weight, dtype: float64


In [111]:
holding = holding['weight'].set_index('ticker')
holding.loc[['1','999']]

AttributeError: 'Series' object has no attribute 'set_index'

In [110]:
holding.loc['1','999'].loc['weight']

KeyError: '999'

In [107]:
df_result.iloc[[1,2]]['γ0']

KeyError: 'γ0'

## 5. Acknowledgement
- Thanks for ***Professor Jinfan ZHANG*** and ***Teaching Assistants*** for designing this assignment.
- Thanks for ***Teaching Assistants*** and ***Undergraduate Student Teaching Fellows*** for reviewing and grading this submission.

Thank you so much!